In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
def rmsle(actual_column, predicted_column):
    sum=0.0
    for x,y in zip(actual_column,predicted_column):
        if x<0 or y<0: #check for negative values. 
            continue
        p = np.log(y+1)
        r = np.log(x+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted_column))**0.5

In [3]:
data=pd.read_csv("..\\Train.csv")
print(data.head())

le_account_type=LabelEncoder()
data['account_type']=le_account_type.fit_transform(data['account_type'])

le_gender=LabelEncoder()
data['gender']=le_gender.fit_transform(data['gender'])

le_loan_enq=LabelEncoder()
data['loan_enq']=le_loan_enq.fit_transform(data['loan_enq'])

for col in data.drop('ID',1).columns:
    if data[col].skew()>0:
        data[col]=np.sqrt(data[col])
    else :
        data[col]=np.square(data[col])

df2=data.set_index('ID').drop(['loan_enq','personal_loan_active','vehicle_loan_active','personal_loan_closed','vehicle_loan_closed'],1)
df2.head()

df2.investment_4=df2.investment_4.fillna(0)

df2.isna().sum()

X=df2.drop('cc_cons',1)
y=df2['cc_cons']
X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.25, random_state=25)

ID account_type gender  age  region_code  cc_cons_apr  dc_cons_apr  \
0  12554      current      M   35          708     24893.00       378.00   
1  17645      current      M   35          123     18941.62       966.00   
2   7604      current      M   55          802      5678.87      2724.00   
3   1590      current      F   29          867     30489.50      1236.00   
4  16556      current      M   34          802      7441.40      6906.04   

   cc_cons_may  dc_cons_may  cc_cons_jun  ...  debit_count_may  \
0     10288.00     29664.00     16291.40  ...               40   
1     20672.00       287.00      4217.00  ...               78   
2      1964.50      3933.11     23956.25  ...               82   
3     12609.88      9138.14     17521.00  ...               38   
4      4364.00      1939.00      2121.00  ...               50   

   max_credit_amount_may  debit_amount_jun  credit_amount_jun  \
0                24563.0          30051.32            12761.0   
1                23948.0          42119.05            76206.0   
2                10752.0          58373.25            75283.0   
3                12607.0          69851.51            68708.0   
4                25213.0          10071.00            80140.0   

   credit_count_jun  debit_count_jun  max_credit_amount_jun  loan_enq  \
0                 2               65                50836.0         Y   
1                21               63                23226.0         Y   
2                 7                1                27036.0         Y   
3                21               83                43037.0         Y   
4                 8               32                32044.0         Y   

   emi_active  cc_cons  
0     1674.09    20014  
1    13043.34    10173  
2    25375.27    16095  
3     3544.33     7707  
4    12780.44    96408  

[5 rows x 44 columns]

In [11]:
lr=LinearRegression()

In [18]:
cols=X.columns
n=len(cols)

iter_col=[]
rmsle_list=[]
for i in range(n):
    iter_col.append(cols[i])
    print(iter_col)
    lr.fit(X_train[iter_col], y_train)
    y_train_pred=lr.predict(X_train[iter_col])
    best_rmsle=rmsle(y_train_pred, y_train)
    rmsle_list.append(best_rmsle)
    print(rmsle_list[-1])
    
min(rmsle_list)

In [40]:
lr.fit(X_train, y_train)
train_pred=lr.predict(X_train)
val_pred=lr.predict(X_val)
train_rmsle=rmsle(train_pred, y_train)
val_rmsle=rmsle(val_pred, y_val)
print(train_rmsle, val_rmsle)

0.8573973735048162 0.8667702059598833


In [44]:
test=pd.read_csv('..\\Test.csv')
test.head()

test['account_type']=le_account_type.transform(test['account_type'])

test['gender']=le_gender.transform(test['gender'])

test['loan_enq']=le_loan_enq.transform(test['loan_enq'])

for col in test.drop('ID',1).columns:
    if test[col].skew()>0:
        test[col]=np.sqrt(test[col])
    else:
        test[col]=np.square(test[col])

X_test=test.set_index('ID').drop(['loan_enq','personal_loan_active','vehicle_loan_active','personal_loan_closed','vehicle_loan_closed'],1)

X_test['investment_4']=X_test.fillna(0)

C:\Users\AP hatela\Miniconda\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [45]:
y_pred=lr.predict(X_test)
y_pred**2

array([30570.02431664, 32371.69032086, 32095.68161125, ...,
       26083.93918803, 27024.10852273, 32240.96191847])

In [46]:
X_test['cc_cons']=y_pred**2

result4=pd.DataFrame(X_test['cc_cons'])

print(result4.head())

result4.to_csv('submission4.csv',index=True)

            cc_cons
ID                 
17591  30570.024317
13541  32371.690321
13431  32095.681611
8687   38768.014784
14727  32103.588421
